# Weil Institute Coding Challenge

Kate Weber

March 7, 2022

kate@kateweber.com

## The Request

Observations every 12 hours for each of the patients in the cohort throughout the month of June 2000. For each of these observations, we need the following data:

* The ratio of discharged patients over the previous 4 hours to the number of nurses in the patient’s unit
* The ratio of nurses to patients in the patient’s unit
* The patient’s latest Tropinin lab values
*  The patient’s latest B-type natriuretic peptide (BNP) lab value (LOINC 33762-6)

## The Deliverable
CSV Data file with one row per patient per observation period

In [50]:
import pandas as pd
import sqlite3
import datetime

# Set up the database connection
con = sqlite3.connect("data/DE_Challenge_DB.sqlite")

# build scaffolding for the snapshots
scaffold = pd.date_range(start='2000-06-01 06:59:00', end='2000-06-30 23:59:00', freq='12h')

## Building out the Unit, Population and Staffing Information

Key measures that must accompany patient data consider the overall environment of the hospital at the snapshot times of 6:59 AM and 6:59 PM every day during the study period.  We start with date scaffolding that will help us gather information for each date/time of interst.  Here, we gather information that will help us understand
* where each patient is at the time of each of the snapshots (`snap_df`),
* who is on the nursing team for each patient at the snapshot time (`care_df`) and
* counts of discharges for each patient at the snapshot time (`discharge_df`)

In [51]:
snapdata = []
careteam = []
discharges = []

# collect data at each of the given snapshot points
for snapdate in scaffold:

    # where is each patient right now?
    patient_location = pd.read_sql_query(f"""
            SELECT SUBJECT_ID,
            CURR_CAREUNIT as UNIT
            FROM TRANSFERS WHERE INTIME <= '{snapdate}'
            AND OUTTIME > '{snapdate}'
        """, con = con)
    patient_location['snapshot_time'] = snapdate
    snapdata.append(patient_location)

    # who is the nursing team for each patient right now?
    care_detail = pd.read_sql_query(f"""
            SELECT SUBJECT_ID, tt.CGID, LABEL, DESCRIPTION
            FROM TREATMENT_TEAM tt
            LEFT JOIN CARE_GIVERS cg on tt.CGID = cg.CGID
            WHERE tt.STARTTIME <= '{snapdate}'
            AND tt.ENDTIME > '{snapdate}'
            AND lower(LABEL) = 'rn' OR lower(DESCRIPTION) = 'rn'
        """, con = con)
    care_detail['snapshot_time'] = snapdate
    careteam.append(care_detail)

    # how many patients were discharged from each unit up to four hours before right now?
    discharge_count = pd.read_sql_query(f"""
            SELECT PREV_CAREUNIT as UNIT,
            count(distinct(SUBJECT_ID)) as DISCHARGE_COUNT
            FROM TRANSFERS
            WHERE INTIME > '{snapdate - datetime.timedelta(hours = 4)}'
              AND INTIME <= '{snapdate}'
             AND EVENTTYPE = 'discharge'
            GROUP BY PREV_CAREUNIT
        """, con = con)
    discharge_count['snapshot_time'] = snapdate
    discharges.append(discharge_count)

We generate pandas dataframes from the gathered information

In [52]:
snap_df = pd.concat(snapdata)
care_df = pd.concat(careteam)
discharge_df = pd.concat(discharges).groupby(['snapshot_time','UNIT'],dropna=False).sum()

## Review and Reshaping Data

Here, we have a df showing where each patient was at a given time.

In [53]:
snap_df.head()

,SUBJECT_ID,UNIT,snapshot_time
0,124,None,2000-06-01 06:59:00
1,6,SICU,2000-06-01 06:59:00
2,109,None,2000-06-01 06:59:00
3,110,NWARD,2000-06-01 06:59:00
4,223,None,2000-06-01 06:59:00


Here, we have one row per patient per RN per careteam

In [54]:
care_df.head()

,SUBJECT_ID,CGID,LABEL,DESCRIPTION,snapshot_time
0,30593,19891.0,RN,RN,2000-06-01 06:59:00
1,30593,17735.0,RN,RN,2000-06-01 06:59:00
2,30593,15526.0,RN,RN,2000-06-01 06:59:00
3,30300,20808.0,RN,RN,2000-06-01 06:59:00
4,30593,20063.0,RN,RN,2000-06-01 06:59:00


For the sake of working data, we'll keep the discharge table in long format but here is an overview of it pivoted out by Unit.

In [55]:
discharge_df.reset_index().pivot(index ='snapshot_time', columns='UNIT', values = 'DISCHARGE_COUNT').fillna(0).head()

UNIT,NaN,CCU,CSRU,MICU,NICU,NWARD,SICU,TSICU
snapshot_time,,,,,,,,
2000-06-01 06:59:00,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2000-06-01 18:59:00,58.0,3.0,0.0,3.0,4.0,4.0,1.0,0.0
2000-06-02 06:59:00,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2000-06-02 18:59:00,49.0,3.0,0.0,8.0,4.0,1.0,2.0,1.0
2000-06-03 06:59:00,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [56]:
discharge_df.head()

DISCHARGE_COUNT
snapshot_time       UNIT                 
2000-06-01 06:59:00 MICU                5
                    NaN                 1
2000-06-01 18:59:00 CCU                 3
                    MICU                3
                    NICU                4

The Unit Population is a rollup of distinct SUBJECT_IDs in a given unit at each snapshot time.

In [57]:
unit_population = snap_df.groupby(['snapshot_time','UNIT'],dropna=False)\
    .nunique()\
    .rename(columns={'SUBJECT_ID':'PATIENT_COUNT'})

For review, here is the information pivoted by UNIT. For the actual patient analysis, we'll keep the `unit_population` table in long form.

In [58]:
unit_population.reset_index().pivot(index ='snapshot_time', columns='UNIT', values = 'PATIENT_COUNT').fillna(0).head()

UNIT,NaN,CCU,CSRU,MICU,NICU,NWARD,SICU,TSICU
snapshot_time,,,,,,,,
2000-06-01 06:59:00,816,67,85,225,220,46,129,84
2000-06-01 18:59:00,766,63,93,204,213,38,124,74
2000-06-02 06:59:00,802,68,91,197,220,43,133,75
2000-06-02 18:59:00,760,66,86,188,221,34,128,76
2000-06-03 06:59:00,803,65,86,193,220,39,126,76


In [59]:
unit_population.head()

PATIENT_COUNT
snapshot_time       UNIT                
2000-06-01 06:59:00 CCU               67
                    CSRU              85
                    MICU             225
                    NICU             220
                    NWARD             46

Now we can work backwards, since we know who was in each unit at a given time and work out how many unique RNs were associated with the unit at that time

In [60]:
caregiver_population = snap_df.set_index(['SUBJECT_ID', 'snapshot_time'])\
    .join(care_df.set_index(['SUBJECT_ID', 'snapshot_time']))\
    .dropna(subset = ['CGID'], axis=0)\
    .reset_index()\
    .drop(columns=['SUBJECT_ID','LABEL', 'DESCRIPTION'])\
    .groupby(['snapshot_time','UNIT'], dropna = False)\
    .nunique().rename(columns={'CGID':'TEAM_COUNT'})
caregiver_population.head(20)

TEAM_COUNT
snapshot_time       UNIT             
2000-06-01 06:59:00 CCU           119
                    CSRU          143
                    MICU          287
                    NICU           90
                    NWARD           8
                    SICU          207
                    TSICU         154
                    NaN           353
2000-06-01 18:59:00 CCU           121
                    CSRU          161
                    MICU          271
                    NICU           83
                    NWARD           6
                    SICU          205
                    TSICU         158
                    NaN           357
2000-06-02 06:59:00 CCU           121
                    CSRU          164
                    MICU          266
                    NICU           84

One more time, just showing this wide but we'll work with the data long.

In [61]:
caregiver_population.reset_index().pivot(index ='snapshot_time', columns='UNIT', values = 'TEAM_COUNT').fillna(0).head()

UNIT,NaN,CCU,CSRU,MICU,NICU,NWARD,SICU,TSICU
snapshot_time,,,,,,,,
2000-06-01 06:59:00,353,119,143,287,90,8,207,154
2000-06-01 18:59:00,357,121,161,271,83,6,205,158
2000-06-02 06:59:00,354,121,164,266,84,4,193,150
2000-06-02 18:59:00,351,103,158,270,70,5,192,159
2000-06-03 06:59:00,350,101,164,286,87,6,195,154


I'm not dropping the NA unit yet because I keep this sort of thing around until the last minute in case someone changes their mind

In [62]:
unit_measures = pd.concat([unit_population, discharge_df, caregiver_population], axis = 1).fillna(0)
unit_measures['discharge_nurse_ratio'] = round(unit_measures.DISCHARGE_COUNT / unit_measures.TEAM_COUNT, 4)
unit_measures['nurse_patient_ratio'] = round(unit_measures.TEAM_COUNT / unit_measures.PATIENT_COUNT, 4)
unit_measures.drop(columns=['PATIENT_COUNT', 'DISCHARGE_COUNT', 'TEAM_COUNT'], inplace=True)
unit_measures.head(20)

discharge_nurse_ratio  nurse_patient_ratio
snapshot_time       UNIT                                             
2000-06-01 06:59:00 CCU                   0.0000               1.7761
                    CSRU                  0.0000               1.6824
                    MICU                  0.0174               1.2756
                    NICU                  0.0000               0.4091
                    NWARD                 0.0000               0.1739
                    SICU                  0.0000               1.6047
                    TSICU                 0.0000               1.8333
                    NaN                   0.0028               0.4326
2000-06-01 18:59:00 CCU                   0.0248               1.9206
                    CSRU                  0.0000               1.7312
                    MICU                  0.0111               1.3284
                    NICU                  0.0482               0.3897
                    NWARD                 0.6667               0.1579
                    SICU                  0.0049               1.6532
                    TSICU                 0.0000               2.1351
                    NaN                   0.1625               0.4661
2000-06-02 06:59:00 CCU                   0.0000               1.7794
                    CSRU                  0.0061               1.8022
                    MICU                  0.0038               1.3503
                    NICU                  0.0000               0.3818

## Building out the cohort's test result data

Now, we go get all the patients in our cohort:

In [63]:
patient_cohort = pd.read_sql_query("""
        SELECT DISTINCT june_admits.SUBJECT_ID,
                june_admits.DOB,
                june_admits.GENDER -- 228 rows
        FROM (SELECT DISTINCT T.SUBJECT_ID,
                              HADM_ID,
                              INTIME,
                              pts.GENDER,
                              pts.DOB
              FROM TRANSFERS T
                INNER JOIN (
                              SELECT SUBJECT_ID, GENDER, min(DOB) as DOB
                              FROM PATIENTS
                              WHERE EXPIRE_FLAG = 0
                              group by SUBJECT_ID, GENDER
                          ) pts
                                   on T.SUBJECT_ID = pts.SUBJECT_ID
                          WHERE T.EVENTTYPE = 'admit'
                            AND INTIME BETWEEN '2000-06-01' AND '2000-07-01'
                            AND (julianday(T.INTIME) - julianday(pts.DOB)) / 365 >= 18) june_admits -- 2402 pts
        INNER JOIN
             (SELECT distinct SUBJECT_ID, HADM_ID
                  FROM LABS
                           INNER JOIN ICD_LABS ON LABS.ITEMID = ICD_LABS.ITEMID
                      AND ICD_LABS.LOINC_CODE in ('6598-7', '10839-9')
                      and LABS.VALUENUM is not NULL) tropo -- 1784 pts (of any age, any admit)
                 ON june_admits.SUBJECT_ID = tropo.SUBJECT_ID
                     AND june_admits.HADM_ID = tropo.HADM_ID
        INNER JOIN
             (SELECT distinct SUBJECT_ID, HADM_ID
              from DIAGNOSES
              WHERE (SUBSTR(DIAGNOSES.ICD9_CODE, 1, 3) like '428%')) dx -- 13,608 pts (of any age, any admit)
               ON june_admits.SUBJECT_ID = dx.SUBJECT_ID
               AND june_admits.HADM_ID = dx.HADM_ID;
            """, con = con)

In [64]:
total_patients = con.execute("SELECT COUNT(DISTINCT SUBJECT_ID) from PATIENTS").fetchone()[0]
print(f'''Total Number of Patients in Dataset: {total_patients}. Patient cohort size: {len(patient_cohort)}
Cohort represents {len(patient_cohort)/total_patients:.1%} of all patients in dataset.''')

Total Number of Patients in Dataset: 46488. Patient cohort size: 228
Cohort represents 0.5% of all patients in dataset.


## Get the most recent results from selected labs before a specific threshold date

In [65]:
def collect_latest_measure(lab_df, pt, threshold_date):
    lab_df = lab_df.loc[lab_df['SUBJECT_ID']==pt]
    # We want to collect a row when there's no data so we fully populate the grid of patient/scaffolding
    # We can always drop the information later
    # So we send back None as a signal for the calling code to build a dummy row
    if len(lab_df) == 0:
        return None
    else:
        try:
            latest_index = lab_df.loc[lab_df['CHARTTIME'] < threshold_date]['CHARTTIME'].idxmax()
            return dict(lab_df.loc[latest_index])
        except ValueError:
            return None


In [66]:
cohort_labs = []
tests = {'tropinin': ['6598-7', '10839-9'],
         'BNP': ['33762-6', '33762-6']} # hackery, I know. makes it possible to swap in 'tuple(tests[foo])' in the query

for test in tests:
    patient_labs = pd.read_sql_query(f"""SELECT LABS.SUBJECT_ID, CHARTTIME, VALUENUM, VALUEUOM, LABEL FROM LABS
                       INNER JOIN ICD_LABS ON LABS.ITEMID = ICD_LABS.ITEMID AND ICD_LABS.LOINC_CODE in {tuple(tests[test])}
                       WHERE SUBJECT_ID IN {tuple(patient_cohort['SUBJECT_ID'])}
                       AND LABS.VALUENUM IS NOT NULL""", con = con, parse_dates=['CHARTTIME'])
    patient_labs['test_family'] = test
    for patient in patient_cohort['SUBJECT_ID']:
        for snapdate in scaffold:
            latest_test = collect_latest_measure(patient_labs, patient, snapdate)
            if latest_test is None:
                latest_test = {'SUBJECT_ID':patient,
                              'CHARTTIME': None,
                              'VALUENUM': None,
                              'VALUEUOM': None,
                              'LABEL': None,
                              'test_family': test}
            latest_test['snapshot_time'] = snapdate
            cohort_labs.append(latest_test)

This remarkable bit of index mangling helps us drop some NaNs out of the columns. I could have used `dropna()` earlier but I wanted to keep the `NA` columns around so I could check them to make sure it was all still making sense

In [67]:
patient_labs = pd.DataFrame(cohort_labs).pivot(index=['snapshot_time', 'SUBJECT_ID'],
                                columns=[ 'test_family', 'LABEL'],
                                values=['VALUENUM'])
patient_labs.columns = patient_labs.columns.to_flat_index()
patient_labs = patient_labs.iloc[:,[1, 2, 4]]
patient_labs.columns = ['Troponin T', 'Troponin I', 'NTproBNP']

patient_labs.head(20)

Troponin T  Troponin I  NTproBNP
snapshot_time       SUBJECT_ID                                  
2000-06-01 06:59:00 309                NaN         NaN       NaN
                    318                NaN         NaN       NaN
                    518                NaN         NaN       NaN
                    740                NaN         NaN       NaN
                    1018               NaN         NaN       NaN
                    1041               NaN         NaN       NaN
                    1317               NaN         NaN       NaN
                    1417               NaN         NaN       NaN
                    2356               NaN         NaN       NaN
                    2791               NaN         NaN       NaN
                    2897               NaN         NaN       NaN
                    3674               NaN         NaN       NaN
                    3796               NaN         NaN       NaN
                    3874               NaN         NaN       NaN
                    3952               NaN         NaN       NaN
                    3987               NaN         NaN       NaN
                    4096               NaN         NaN       NaN
                    4390               NaN         NaN       NaN
                    4759               NaN         NaN       NaN
                    4787              0.01         NaN       NaN

## Build out Reporting Framework

Now, we build a framework of information with the units and subjects who are relevant at a given point in time, constrained to this cohort.

In [68]:
# letting go of the NA unit here - left-joining from here out will do the job

snap_cohort_df = snap_df[snap_df.SUBJECT_ID.isin(patient_cohort['SUBJECT_ID'])]\
    .dropna(subset='UNIT')\
    .set_index(['snapshot_time','UNIT'])

snap_cohort_df.head(20)

SUBJECT_ID
snapshot_time       UNIT             
2000-06-01 06:59:00 MICU         4787
                    MICU         6440
                    MICU        12567
                    CCU         14522
                    TSICU       94530
2000-06-01 18:59:00 MICU         4787
                    MICU         6440
                    CCU         11672
                    MICU        12567
                    CCU         14522
                    MICU        28698
                    SICU        96518
                    MICU        94698
                    TSICU       94530
2000-06-02 06:59:00 MICU         4787
                    MICU         6440
                    SICU        10504
                    CCU         11672
                    MICU        12567
                    CCU         14522

... and then we graft on the unit data and the lab data

In [69]:
wide_data = snap_cohort_df.join(unit_measures, how = 'left')\
            .reset_index()\
            .set_index(['snapshot_time', 'SUBJECT_ID'])\
            .join(patient_labs, how = 'left')\
            .reset_index()\
            .set_index('SUBJECT_ID')\
            .join(patient_cohort.set_index('SUBJECT_ID'), how = 'left')

In [70]:
wide_data

,snapshot_time,UNIT,discharge_nurse_ratio,nurse_patient_ratio,Troponin T,Troponin I,NTproBNP,DOB,GENDER
SUBJECT_ID,,,,,,,,,
309,2000-06-04 18:59:00,CCU,0.0000,2.1569,NaN,NaN,NaN,1938-10-02 00:00:00,M
309,2000-06-05 06:59:00,CCU,0.0000,1.9057,0.34,NaN,NaN,1938-10-02 00:00:00,M
309,2000-06-05 18:59:00,CCU,0.0078,2.3889,0.25,NaN,NaN,1938-10-02 00:00:00,M
318,2000-06-07 18:59:00,CCU,0.0000,2.0784,NaN,29.1,NaN,1937-09-02 00:00:00,M
318,2000-06-08 06:59:00,CCU,0.0000,1.9123,NaN,19.9,NaN,1937-09-02 00:00:00,M
...,...,...,...,...,...,...,...,...,...
98649,2000-06-21 06:59:00,MICU,0.0035,1.1748,0.05,NaN,NaN,1924-07-15 00:00:00,F
98649,2000-06-21 18:59:00,MICU,0.0107,1.2217,0.05,NaN,NaN,1924-07-15 00:00:00,F
98649,2000-06-22 06:59:00,MICU,0.0069,1.2101,0.05,NaN,NaN,1924-07-15 00:00:00,F


## Write it out

Finally, we clean up a bit and send this table to CSV

In [71]:
wide_data.columns = [x.upper().replace(' ', '_') for x in wide_data.columns]

In [72]:
wide_data.to_csv('de_challenge_retrospective.csv', date_format="%Y-%m-%dT%H:%M:%S-04:00")